<a href="https://colab.research.google.com/github/worachot-n/Text_summarization_T5/blob/main/1_1_Text_Summarization_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
# !pip install sentencepiece
# !pip install transformers
# !pip install rich[jupyter]

In [21]:
import pandas as pd
df = pd.read_csv("BBC_News_Summary.csv")

In [22]:
df.sample(10)

,Unnamed: 0,articles,summaries,categories
414,414,Blair rejects Iraq advice calls\r\n\r\nTony Bl...,Ms Hewitt also announced a new drive to help w...,politics
420,420,Strachan turns down Pompey\r\n\r\nFormer South...,Fifth seed Robin Soderling took the Milan Indo...,sport
1644,1644,Downloads enter US singles chart\r\n\r\nDigita...,Bridget Jones: The Edge of Reason was named Ev...,entertainment
416,416,Chancellor rallies Labour voters\r\n\r\nGordon...,Mr Blair said that whether the public chose Mi...,politics
1232,1232,Sony wares win innovation award\r\n\r\nSony ha...,Mr Raskin was one of the first employees at Ap...,tech
1544,1544,Little Britain two top comic list\r\n\r\nLittl...,It is predicted that the Band Aid 20 song will...,entertainment
1748,1748,Market unfazed by Aurora setback\r\n\r\nAs the...,Shares in US phone company MCI have risen on s...,business
1264,1264,Ask Jeeves joins web log market\r\n\r\nAsk Jee...,"A US woman is suing Hewlett Packard (HP), sayi...",tech
629,629,Yelling takes Cardiff hat-trick\r\n\r\nEuropea...,Day has been displaced by the arrival of Simon...,sport
1043,1043,Cyber crime booms in 2004\r\n\r\nThe last 12 m...,The Gameboyzz Orchestra Project has taken the ...,tech


In [23]:
df_business = df.loc[df['categories'] == 'business']

In [24]:
df_business = df_business[['articles', 'summaries']]
df_business

,articles,summaries
1715,EU 'too slow' on economic reforms\r\n\r\nMost ...,Senator Kerry attacked President Bush's econom...
1716,Glazer makes new Man Utd approach\r\n\r\nMalco...,A slimming aid made from a southern African ca...
1717,Five million Germans out of work\r\n\r\nGerman...,Job creation was one of last year's main conce...
1718,Axa Sun Life cuts bonus payments\r\n\r\nLife i...,Malcolm Glazer has made a fresh approach to bu...
1719,Amex shares up on spin-off news\r\n\r\nShares ...,"If admitted into the EU, Turkey would contribu..."
...,...,...
2220,GE sees 'excellent' world economy\r\n\r\nUS be...,The WTO said that many developing countries su...
2221,Millions 'to lose textile jobs'\r\n\r\nMillion...,Reuters news agency reported that Iraq's Oil M...
2222,US to rule on Yukos refuge call\r\n\r\nYukos h...,Parmalat has sued 45 banks as it tries to recl...
2223,Business confidence dips in Japan\r\n\r\nBusin...,Bombardier said restructuring plans drawn up b...


In [25]:
df_business = df_business.replace(r'\n\n',' ', regex=True)
df_business

,articles,summaries
1715,EU 'too slow' on economic reforms\r\n\r\nMost ...,Senator Kerry attacked President Bush's econom...
1716,Glazer makes new Man Utd approach\r\n\r\nMalco...,A slimming aid made from a southern African ca...
1717,Five million Germans out of work\r\n\r\nGerman...,Job creation was one of last year's main conce...
1718,Axa Sun Life cuts bonus payments\r\n\r\nLife i...,Malcolm Glazer has made a fresh approach to bu...
1719,Amex shares up on spin-off news\r\n\r\nShares ...,"If admitted into the EU, Turkey would contribu..."
...,...,...
2220,GE sees 'excellent' world economy\r\n\r\nUS be...,The WTO said that many developing countries su...
2221,Millions 'to lose textile jobs'\r\n\r\nMillion...,Reuters news agency reported that Iraq's Oil M...
2222,US to rule on Yukos refuge call\r\n\r\nYukos h...,Parmalat has sued 45 banks as it tries to recl...
2223,Business confidence dips in Japan\r\n\r\nBusin...,Bombardier said restructuring plans drawn up b...


In [26]:
df_business["summaries"] = "summarize: "+df_business["summaries"]

In [27]:
df_business.head()

,articles,summaries
1715,EU 'too slow' on economic reforms\r\n\r\nMost ...,summarize: Senator Kerry attacked President Bu...
1716,Glazer makes new Man Utd approach\r\n\r\nMalco...,summarize: A slimming aid made from a southern...
1717,Five million Germans out of work\r\n\r\nGerman...,summarize: Job creation was one of last year's...
1718,Axa Sun Life cuts bonus payments\r\n\r\nLife i...,summarize: Malcolm Glazer has made a fresh app...
1719,Amex shares up on spin-off news\r\n\r\nShares ...,"summarize: If admitted into the EU, Turkey wou..."


In [28]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)

In [29]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [30]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [31]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [32]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [33]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [34]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":4,          # training batch size
    "VALID_BATCH_SIZE":4,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 
}

In [35]:
T5Trainer(dataframe=df_business[:500], source_text="articles", target_text="summaries", model_params=model_params, output_dir="outputs")

[18:00:46] [Model]: Loading t5-base...                     <ipython-input-33-b1eef7f6be59>:14

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 6.00 GiB total capacity; 4.42 GiB already allocated; 0 bytes free; 4.52 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF